         ECOLE NATIONALE DE LA STATISTIQUE ET DE L'ANALYSE ECONOMIQUE PIERRE NDIAYE     
            COURS DE STATISTIQUES EXPLORATOIRE ET SPATIALE - ISE1_CYCLE LONG          
                            ENSEIGNANT: MR HEMA                                     
                      TP5_GOOGLE EARTH ENGINE AVEC PYTHON                            
                     MEMBRES: FOGWOUNG DJOUFACK SARAH-LAURE                         
                              NGUEMFOUO NGOUMTSA CELINA                            
                              NIASS AHMADOU                                       
                              SENE MALICK                                        

  Consigne du TP :
  
  - Importer les fichiers
  - Visualiser le raster Worldpop Sénégal 
  - Calculer le nombre de personnes par admin et exporter sous format .csv
  - Ramener le raster population à 5km
  - Visualiser le nouveau raster de la population avec un choix de couleur approprié
  - A partir du nouveau raster de la population, calculer un nouveau raster en le multipliant par 0.1%(qui est supposé représenté le nombre d'enfants de 0 à 12 ans)
  - Compter le nombre d'enfants dans chacune des classifications qu'on avait avec la raster de la malaria
  - Calculer le nombre d'enfants atteints de malaria par admin(0-3) et exporter
  - Calcul du taux d'enfants atteints de malaria par admin (0-3)


In [2]:
############   STEP1: Importation des packages   ############

import ee  # c'est la bibliothèque Python qui permet d'accéder à l'API de Google Earth Engine
import geemap # Elle permet de créer et visualiser des cartes interactives basées sur des données GEE.
import pandas as pd # Elle permet la manipulation et l’analyse généralement pour des données tabulaires
from ipyleaflet import ImageOverlay # Permet d'ajouter des images en superposition (overlay) sur une carte interactive Leaflet.
from ipyleaflet import WidgetControl # Toujours pour une meilleure visualisation des cartes: permettant d’ajouter des widgets interactifs tels des boutons et champs de texte; sur des cartes.
from ipywidgets import VBox, HTML
# Toujours pour l'aspect visuel,VBox permet d'organiser en colonne des éléments qu'on aura empiler en parametres dans VBox
# et HTML c'est juste pour la mise en forme des textes (couleurs, gras, italique ...)


# Authentification et initialisation Google Earth Engine
ee.Authenticate()  # Ceci permet de gérer le processus d'authentification pour accéder à Google Earth Engine.
ee.Initialize()   # Initialise l'instance Earth Engine pour permettre certaines opérations qu'on fera par la suite

print("DONE") # Juste pour savoir quand ca aura finit d'éxecuter 

Enter verification code:  4/1AeanS0b0U--MVRReEzpfCCx7eM-IDHPmUSXshqrIPXOoIB_4hhl1z8PTKkE



Successfully saved authorization token.
DONE


In [3]:
############   STEP2: Importation des données vectorielles - caractéristiques et visualisation  ############

# Créons une carte interactive pour afficher les données vectorielles et raster
Map = geemap.Map() # Initialise un objet de carte interactive pour afficher les couches de données géospatiales.

# Importons et visualisons les données sur la population
# Chargeons les différents niveaux administratifs
pays = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/geoBoundaries-NER-ADM0")
regions = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/geoBoundaries-NER-ADM1")
departements = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/geoBoundaries-NER-ADM2")
communes = ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/geoBoundaries-NER-ADM3")

# Calculs statistiques : récupérer et afficher des informations pour le niveau pays
country_count = pays.size().getInfo()  # Obtenir le nombre de géométries dans le shapefile des pays
print(f'Nombre de géométries (pays) : {country_count}')

crs_country = pays.first().geometry().projection().getInfo()  # Obtenir le système de coordonnées (CRS) du shapefile
print(f'CRS du shapefile du pays : {crs_country}')

extent_country = pays.geometry().bounds().getInfo()  # Obtenir les limites géographiques du shapefile
print(f'Extent (limites) du pays : {extent_country}')

# Calculs statistiques : récupérer et afficher des informations pour le niveau régions
regions_count = regions.size().getInfo()  # Obtenir le nombre de géométries dans le shapefile des pays
print(f'Nombre de géométries (regions) : {regions_count}')

# Visualisation des shapefiles : ajouter le shapefile du niveau pays sur la carte
Map.addLayer(pays, {}, 'Pays')  # Ajouter le layer du pays sans style spécifique
Map.addLayer(regions, {}, 'Regions') # Ajouter le layer des régions sans style spécifique
Map.centerObject(pays, 6)  # Centrer la vue sur le shapefile du pays avec un niveau de zoom de 6
Map

Nombre de géométries (pays) : 1
CRS du shapefile du pays : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Extent (limites) du pays : {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[0.16666781762811478, 11.691999274801383], [15.996668338772286, 11.691999274801383], [15.996668338772286, 23.517178579212032], [0.16666781762811478, 23.517178579212032], [0.16666781762811478, 11.691999274801383]]]}
Nombre de géométries (regions) : 6


Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
############   STEP3: Données Raster - Visualisation de la population  ############
# Charger un raster de population (ex. WorldPop ou une source de données de population pour le Niger)
population_raster = ee.Image("projects/ee-fogwoungsarahlauretp2/assets/NER_population_v1_0_gridded")

# Ajouter le raster de population à la carte avec une palette de couleurs pour représenter les densités de population
palette = ['ffffcc', 'c2e699', '78c679', '31a354', '006837']  # Palette de couleurs allant du clair au foncé pour illustrer les variations de densité
Map.addLayer(population_raster, {'palette': palette, 'min': 0}, 'Population')  # Définir la palette et les valeurs min de population

# Vérifier la résolution spatiale du raster en obtenant la taille des pixels
pixel_size = population_raster.projection().nominalScale().getInfo()  # Obtenir la résolution (taille des pixels en mètres)
print(f"Taille des pixels du raster de population : {pixel_size} mètres")

# Afficher la carte combinée (données vectorielles + données raster)
Map  # Affiche la carte interactive avec les couches de données ajoutées


Taille des pixels du raster de population : 92.76624195800512 mètres


Map(bottom=7664.0, center=[17.895114303749143, 10.53595264651163], controls=(WidgetControl(options=['position'…

In [5]:
############   STEP4: CALCUL DE LA POPULATION AVEC Taille de pixel= 100m  ############
# Fonction pour calculer la population dans une zone administrative donnée
def calculer_population_par_zone(admin_level, zone_name):
    pop_totale = population_raster.reduceRegion(
        reducer=ee.Reducer.sum(),  # Utilisation du réducteur de somme pour agréger la population dans la zone
        geometry=admin_level.geometry(),  # Définition de la géométrie de la zone administrative
        scale=100,  # Échelle basée sur la résolution du raster de population
        maxPixels=1e13  # Limite de pixels pour la réduction
    ).get('b1')  # 'b1' correspond à la bande de population par défaut dans WorldPop

    return ee.Feature(None, {'ID': zone_name, 'Population': pop_totale})  # Création d'une Feature avec l'ID de la zone et la population calculée

# Calculer la population pour chaque niveau administratif en appliquant la fonction sur chaque Feature
pop_pays = pays.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par pays
pop_regions = regions.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par région
pop_departements = departements.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par département
pop_communes = communes.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par commune

# Fonction pour convertir une FeatureCollection en dictionnaire avec les ID et les populations
def features_to_dict(feature_collection):
    ids = feature_collection.aggregate_array('ID').getInfo()  # Extraction des IDs de chaque Feature
    populations = feature_collection.aggregate_array('Population').getInfo()  # Extraction des valeurs de population
    return [{'ID': id, 'Population': pop} for id, pop in zip(ids, populations)]  # Construction d'une liste de dictionnaires avec les données

# Obtenir les données sous forme de liste de dictionnaires pour chaque niveau administratif
pop_pays_data = features_to_dict(pop_pays)  # Données pour les pays
pop_regions_data = features_to_dict(pop_regions)  # Données pour les régions
pop_departements_data = features_to_dict(pop_departements)  # Données pour les départements
pop_communes_data = features_to_dict(pop_communes)  # Données pour les communes

# Convertir les résultats en DataFrames pandas et les exporter en fichiers CSV
pop_pays_df = pd.DataFrame(pop_pays_data)  # DataFrame pour les pays
pop_regions_df = pd.DataFrame(pop_regions_data)  # DataFrame pour les régions
pop_departements_df = pd.DataFrame(pop_departements_data)  # DataFrame pour les départements
pop_communes_df = pd.DataFrame(pop_communes_data)  # DataFrame pour les communes

# Exportation de chaque DataFrame au format CSV
pop_pays_df.to_csv('population_par_pays.csv', index=False)  # Export des données de population par pays
pop_regions_df.to_csv('population_par_regions.csv', index=False)  # Export des données de population par régions
pop_departements_df.to_csv('population_par_departements.csv', index=False)  # Export des données de population par départements
pop_communes_df.to_csv('population_par_communes.csv', index=False)  # Export des données de population par communes

# Afficher les populations par niveau administratif dans la console
print("Population par pays:", pop_pays_data)
print("Population par régions:", pop_regions_data)
#print("Population par départements:", pop_departements_data)
#print("Population par communes:", pop_communes_data)

# Message de confirmation
print("Les fichiers CSV de population par niveau administratif ont été exportés avec succès.")


Population par pays: [{'ID': '22259449B93083360167036', 'Population': 20595207.982590064}]
Population par régions: [{'ID': '87150794B58281898249288', 'Population': 4488225.203142479}, {'ID': '87150794B47488127174201', 'Population': 2459948.256106143}, {'ID': '87150794B29979307394387', 'Population': 1132975.1507399175}, {'ID': '87150794B24708462658640', 'Population': 3337077.2957259794}, {'ID': '87150794B5839560144768', 'Population': 4215028.940165879}, {'ID': '87150794B17606801029036', 'Population': 5039519.390644689}]
Les fichiers CSV de population par niveau administratif ont été exportés avec succès.


On est entrain d'aller à une taille de pixel beaucoup plus grande que celle qu'on avait. 
And remember what the teacher told when we were doing flexible scale: Grande taille de pixels
implique plus basse résolution et donc moins de précision et de finesse et par conséquent, perte
d'informations. 


In [6]:
############   STEP4: CALCUL DE LA POPULATION AVEC Taille de pixel= 5000m  ############
# Définir la nouvelle résolution pour l'agrégation en mètres (ici, 5 km = 5000 mètres)
nouvelle_resolution = 5000

# Agréger les pixels du raster de population à la nouvelle résolution définie sans changer la population
population_raster_5km = population_raster.reduceResolution(
    reducer=ee.Reducer.sum(),  # Utiliser la somme pour agréger les valeurs de population dans chaque zone
    maxPixels=65000
).reproject(
    crs=population_raster.projection(),  # Conserver le système de coordonnées d'origine du raster
    scale=nouvelle_resolution  # Appliquer la nouvelle résolution de 5 km
)

# Calcul de la population totale avant et après redimensionnement pour vérifier que la population reste inchangée
population_totale_initiale = population_raster.reduceRegion(
    reducer=ee.Reducer.sum(),  # Agréger toutes les valeurs de population
    geometry=population_raster.geometry(),  # Zone géographique du pays ou de la zone d'étude
    scale=100,  # Résolution d'origine du raster
    maxPixels=1e13  # Limite de pixels pour éviter la surcharge de calcul
).get('b1')

population_totale_5km = population_raster_5km.reduceRegion(
    reducer=ee.Reducer.sum(),  # Agréger toutes les valeurs de population après redimensionnement
    geometry=population_raster.geometry(),  # Zone géographique
    scale=nouvelle_resolution,  # Résolution de 5 km
    maxPixels=1e13  # Limite de pixels pour éviter la surcharge de calcul
).get('b1')

# Afficher la population totale avant et après redimensionnement
print("Population totale initiale :", population_totale_initiale.getInfo())
print("Population totale après redimensionnement :", population_totale_5km.getInfo())


Population totale initiale : 20733331.108510993
Population totale après redimensionnement : 1242.0289919135337


ANOTHER WAY OF DOING: On utilisera des sommes pondérées 
REMEMBER THE GOAL: L'objectif est de calculer la population totale en considérant la nouvelle résolution (5 km) en partant de petits pixels de 100 m (notre résolution originale) , et s'assurer qu'on ne perd plus trop d'informations comme précédemment. Donc on doit s'assurer que la population dans chaque "grande zone" ( celles de 5km) est agrégée de manière juste, c'est-à-dire que chaque pixel contribue à la somme en fonction de sa taille et de sa densité de population.
EXPLICATION STEP BY STEP DE LA METHODE RESAMPLE BICUBIC QU'ON VA UTILISER: 
- Dans un premier temps, nous utilisons la méthode reduceResolution avec l'agrégateur ee.Reducer.sum() pour réduire la résolution du raster de 100m à 5 km: Cela signifie que plusieurs petits pixels (100 m) vont être combinés en un seul pixel de 5 km, toutefois  ce stde, les poids ou proportions de chaque pixel ne sont pas encore pris en compte.
- Création d'un raster qui prendra en compte le poids de chaque pixel de 100m dans le grand pixel de 5km, puisqu'on sait que chaque pixel de 100m occupe une partie proportionnelle de la zone totale de 5 km: on utilisera la méthode (resample('bicubic'))
- Pondération de la population avec les poids précédemment calculés: chaque valeur de population dans le raster de 100m est multipliée par le poids correspondant dans le poids_raster.
- Une fois que chaque pixel de 100m a été multiplié par son poids, nous utilisons reduceRegion avec un réducteur de somme (ee.Reducer.sum()) pour agréger toutes les valeurs pondérées et obtenir la population totale dans chaque zone de 5 km. 

In [7]:
# Définir la nouvelle résolution (ici 5 km)
nouvelle_resolution = 5000  # 5 km

# 2. Maintenant, pour ajuster la population en fonction des poids, nous allons créer un raster qui
# représente la surface de chaque pixel de 100m dans une zone de 5 km.
# On calcule la proportion de chaque pixel dans la zone de 5 km.
# Calculer la surface du pixel dans chaque zone de 5 km (le poids est basé sur la surface relative).

# Créer un raster de poids basé sur la superficie des zones
poids_raster = population_raster.resample('bicubic').reproject(
    crs=population_raster.projection(), 
    scale=nouvelle_resolution  # La même résolution de 5 km
)

# 3. Appliquer le poids au raster de population. Cela va multiplier la population de chaque pixel par le poids
# C'est une forme de pondération basée sur la surface.
population_raster_poids = population_raster.multiply(poids_raster)

# 4. Agréger les valeurs pondérées pour obtenir la population totale agrégée dans la zone de 5 km.
population_totale_ponderee = population_raster_poids.reduceRegion(
    reducer=ee.Reducer.sum(),  # Faire la somme de la population pondérée
    geometry=population_raster.geometry(),  # Zone géographique
    scale=nouvelle_resolution,  # La résolution de 5 km
    maxPixels=1e13  # Limite de pixels pour éviter la surcharge
)

# 5. Afficheons le résultat avant agrégation
population_totale_initiale = population_raster.reduceRegion(
    reducer=ee.Reducer.sum(),  # Agréger toutes les valeurs de population
    geometry=population_raster.geometry(),  # Zone géographique
    scale=100,  # Résolution originale du raster
    maxPixels=1e13  # Limite de pixels
).get('b1')

# Afficher les résultats
print("Population totale initiale :", population_totale_initiale.getInfo())
print("Population totale après agrégation pondérée :", population_totale_ponderee.get('b1').getInfo())

# Visualisation du raster obtenu
visualization_params = {
    'min': 0,
    'max': 5000,  # Ajuster en fonction des valeurs de population dans les zones agrégées
    'palette': ['blue', 'yellow', 'orange', 'red']
}


Population totale initiale : 20733331.108510993
Population totale après agrégation pondérée : 249010.59662366012


TOUJOURS PAS BON, PERTE ENORME DE DONNEES 
UTILISONS LA METHODE TROUVEE PAR LE GROUPE 3

In [18]:
### AJOUT DE BUFFER AU RASTER (On ajoute 50 pixels de 100m au raster population_raster)

# Obtenir la projection et la résolution du raster
proj = population_raster.projection()
scale = 100  # Résolution (100 m dans ton cas)

# Distance pour 50 pixels supplémentaires
buffer_distance = 5000  # 50 pixels * 100 m = 5000 m

# Étendre les limites ("bounding box") du raster
geometrie = population_raster.geometry()  
extended_boundaries = geometrie.buffer(buffer_distance)  # Ajouter 5000 m de chaque côté

# Créer un raster vide avec les nouvelles limites
raster_vide = ee.Image(0).rename(['band1']).reproject(crs=proj, scale=scale).clip(extended_boundaries)

# Fusionner le raster d'origine avec le raster étendu
raster_large = raster_vide.where(population_raster.mask(), population_raster)

## RESCALING FROM 100 TO 5000
p_1=proj.atScale(5000)
Raster_5km= raster_large.reduceResolution(
    reducer=ee.Reducer.sum().unweighted(),
    maxPixels= 65000
  ).reproject(
    crs= p_1
  ).clip(pays)
# Calcul de la population totale après  redimensionnement pour vérifier que la population reste inchangée
population_raster_5km = Raster_5km.clip(pays).reduceRegion(
    reducer=ee.Reducer.sum(),  # Agréger toutes les valeurs de population
    geometry=pays.geometry(),  # Zone géographique du pays ou de la zone d'étude
    scale=5000,  # Résolution d'origine du raster
    maxPixels=1e13  # Limite de pixels pour éviter la surcharge de calcul
).get('band1')

population_totale_5km =round(population_raster_5km.getInfo(), 0)
print("Population totale obtenue à 5km :",population_totale_5km)
## COMPARAISON AVEC LA VALEUR INITIALE 
print("Population totale initiale :", population_totale_initiale.getInfo())

Population totale obtenue à 5km : 20524805.0
Population totale initiale : 20733331.108510993


LA DIFFERENCE N'EST PAS GRANDE DONC NOUS ALLONS CONTINUER AVEC CETTE VALEUR

In [24]:
# Calculer le raster représentant le pourcentage d'enfants âgés de 2 à 10 ans dans la population.
# Hypothèse : les enfants de 2 à 10 ans représentent 0,1 % de la population totale
population_enfants_2_10 = Raster_5km.multiply(0.001)

# Ajouter le raster calculé pour la population d'enfants (2-10 ans) à la carte pour visualisation
# La palette de couleurs et la plage (min et max) sont définies pour représenter les valeurs du pourcentage d'enfants
Map.addLayer(
    population_enfants_2_10, 
    {'palette': palette, 'min': 0, 'max': 3},  # Palette de couleurs et limites de l'échelle pour la population d'enfants
    'Population Enfants (2-10 ans, 5km resolution)'  # Légende pour indiquer la tranche d'âge et la résolution
)

# Calculer le nombre total d'enfants de 2 à 10 ans pour le niveau pays
nombre_total_enfants_2_10 = population_enfants_2_10.reduceRegion(
    reducer=ee.Reducer.sum(),  # Agrégation par somme
    geometry=pays.geometry(),  # Zone géographique
    scale=5000,  # Résolution spatiale de 5 km
    maxPixels=1e13  # Gestion de grandes zones/pixels
).getInfo()

# Afficher le nombre total d'enfants
print("Nombre total d'enfants âgés de 2 à 10 ans au niveau du pays :", round(nombre_total_enfants_2_10['band1']))

# Afficher les résultats
print("Population totale initiale :", population_totale_initiale.getInfo())
# Afficher la carte avec le raster des enfants âgés de 2-10 ans
Map


Nombre total d'enfants âgés de 2 à 10 ans au niveau du pays : 20525
Pourcentage d'enfants âgés de 2 à 10 ans au niveau du pays : 0.1 %
Population totale initiale : 20733331.108510993


Map(bottom=1096.0, center=[37.16031654673677, 18.193359375000004], controls=(WidgetControl(options=['position'…

In [26]:
# Fonction pour obtenir les images raster par année et sélectionner la première bande
def importation_rasters(annee):
    nom_raster = f"202406_Global_Pf_Parasite_Rate_NER_{annee}"
    asset_id = f'projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/{nom_raster}'
    return ee.Image(asset_id).select(0)

# Liste des années pour les rasters
annees = list(range(2000, 2023))

# Charger toutes les images raster et sélectionner la première bande
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection à partir de la liste d'images
stack_raster = ee.ImageCollection(rasters)

# Calculer la moyenne sur la collection
moyenne_raster = stack_raster.mean()

In [27]:
# Fonction pour importer les images raster par année
# ------------------------
# Cette fonction prend une année en paramètre, construit le nom du raster et renvoie l'image raster correspondante
def importation_rasters(annee):
    # Créer le nom du raster en fonction de l'année donnée
    nom_raster = f"202406_Global_Pf_Parasite_Rate_NER_{annee}"
    # Construire l'ID de l'asset dans Google Earth Engine
    asset_id = f'projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/{nom_raster}'
    # Retourner l'image raster en sélectionnant la première bande (index 0)
    return ee.Image(asset_id).select(0)

# ------------------------
# Charger les rasters et créer une collection
# ------------------------
# Liste des années pour les rasters de 2020 à 2022
annees = list(range(2020, 2023))

# Charger toutes les images raster en utilisant la fonction définie précédemment
# Cette liste contient toutes les images raster importées pour chaque année
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection d'images à partir de la liste d'images raster
stack_raster = ee.ImageCollection(rasters)

# Calculer la moyenne et l'écart type des valeurs de pixels pour toutes les images de la collection
moyenne_raster = stack_raster.mean()
ecart_type_raster = stack_raster.reduce(ee.Reducer.stdDev())

# Paramètres de visualisation pour la moyenne et l'écart type
vis_params_moyenne = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']  # Palette de couleurs pour visualiser la moyenne
}

vis_params_ecart_type = {
    'min': 0,
    'max': 0.5,
    'palette': ['white', 'orange', 'red']  # Palette de couleurs pour visualiser l'écart type
}

# Créer une carte pour visualiser la moyenne et l'écart type
Map_stats = geemap.Map(center=[12, -1], zoom=6)  # Initialiser la carte centrée sur des coordonnées spécifiques
Map_stats.addLayer(moyenne_raster, vis_params_moyenne, 'Moyenne Taux Malaria (2000-2022)')  # Ajouter la couche de moyenne
Map_stats.addLayer(ecart_type_raster, vis_params_ecart_type, 'Ecart Type Taux Malaria (2000-2022)')  # Ajouter la couche d'écart type

# Ajouter des barres de couleur pour les deux couches
Map_stats.add_colorbar(vis_params=vis_params_moyenne, label="Moyenne des valeurs des pixels")  # Légende pour la moyenne
Map_stats.add_colorbar(vis_params=vis_params_ecart_type, label="Ecart Type des valeurs des pixels")  # Légende pour l'écart type

# Afficher la carte
Map_stats

Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [36]:
# 2. Représenter la situation de chaque pixel en 2020, 2021 et 2022
# Fonction pour représenter la situation de chaque pixel pour une année donnée
def representer_situation(annee):
    # Importer l'image raster pour l'année spécifiée
    image = importation_rasters(annee)
    
    # Calculer l'indicateur de situation par pixel
    # Catégories : 
    # - Aucune situation grave (valeur < moyenne + écart type)
    # - Situation modérée (moyenne + écart type <= valeur < moyenne + 2 * écart type)
    # - Crise (valeur >= moyenne + 2 * écart type)
    situation = image.expression(
        "(b('b1') < moyenne + ecart_type) ? 1 : (b('b1') < moyenne + 2 * ecart_type) ? 2 : 3",
        {
            'b1': image,
            'moyenne': moyenne_raster,
            'ecart_type': ecart_type_raster
        }
    )
    
    # Clipper l'image à la frontière géographique de la région d'intérêt
    return situation.clip(ee.FeatureCollection("projects/ee-fogwoungsarahlauretp2/assets/TP5_StatistiqueExploratoireSpatiale_ENSAE2024_ISEP3/geoBoundaries-NER-ADM0"))

# Créer une carte pour représenter les situations en 2020, 2021 et 2022
Map_situations = geemap.Map(center=[12, -1], zoom=6)

# Récupérer les images pour les années 2020, 2021 et 2022
situation_2020 = representer_situation(2020)
situation_2021 = representer_situation(2021)
situation_2022 = representer_situation(2022)

# Ajouter les images à la carte avec des paramètres de visualisation
vis_params_situation = {
    'min': 1,
    'max': 3,
    'palette': ['green', 'yellow', 'red']  # 1: aucune situation grave, 2: modérée, 3: crise
}

Map_situations.addLayer(situation_2020, vis_params_situation, 'Situation 2020')
Map_situations.addLayer(situation_2021, vis_params_situation, 'Situation 2021')
Map_situations.addLayer(situation_2022, vis_params_situation, 'Situation 2022')

# Créer un widget HTML pour la légende de la carte
legend_html = """
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>Indicateur de Situation (2020-2022)</h4>
        <p style='margin: 0'><span style='color: green;'>⬤</span> Aucune situation grave</p>
        <p style='margin: 0'><span style='color: yellow;'>⬤</span> Situation modérée</p>
        <p style='margin: 0'><span style='color: red;'>⬤</span> Crise</p>
    </div>
"""
# Créer le widget HTML pour la légende
legend_widget = HTML(value=legend_html)

# Ajouter le widget de légende à la carte
legend_control = WidgetControl(widget=legend_widget, position="topright")
Map_situations.add_control(legend_control)

# Afficher la carte
Map_situations

Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [42]:
# Importer le raster pour l'année 2022
raster_2022 = importation_rasters(2022)

# Calculer l'indicateur de situation par pixel pour 2022
situation_2022 = representer_situation(2022)

# Masquer les autres zones en laissant seulement les taux de malaria pour chaque situation
taux_aucune_situation = raster_2022.updateMask(situation_2022.eq(1))  # Aucune situation grave
taux_situation_moderee = raster_2022.updateMask(situation_2022.eq(2)) # Situation modérée
taux_situation_crise = raster_2022.updateMask(situation_2022.eq(3))  # Crise

# Calculer la population d'enfants pour chaque situation en multipliant les taux par la population d'enfants
# population_enfants_2_10 représente la population totale d'enfants dans la plage d'âge souhaitée
population_enfants_aucune_situation = population_enfants_2_10.multiply(taux_aucune_situation)
population_enfants_situation_moderee = population_enfants_2_10.multiply(taux_situation_moderee)
population_enfants_situation_crise = population_enfants_2_10.multiply(taux_situation_crise)

# Paramètres de visualisation pour les cartes
vis_params_population = {
    'min': 0,
    'max': 50,  # Ajuster selon les valeurs maximales de population dans les données
    'palette': ['white', 'blue']  # Palette pour visualiser la population d'enfants
}

# Créer une carte pour visualiser les populations d'enfants dans chaque situation
Map_population = geemap.Map(center=[12, -1], zoom=6)

# Ajouter les couches pour chaque situation
Map_population.addLayer(population_enfants_aucune_situation, vis_params_population, 'Aucune Situation Grave')
Map_population.addLayer(population_enfants_situation_moderee, vis_params_population, 'Situation Modérée')
Map_population.addLayer(population_enfants_situation_crise, vis_params_population, 'Crise')
print("finish")

finish


In [53]:
# Renommer les bandes pour qu'elles correspondent
population_enfants_aucune_situation = population_enfants_aucune_situation.rename(['b1'])
population_enfants_situation_moderee = population_enfants_situation_moderee.rename(['b1'])
population_enfants_situation_crise = population_enfants_situation_crise.rename(['b1'])

# Fonction pour calculer la population d'enfants dans chaque situation pour un niveau administratif
def calculer_population_enfants_par_zone(admin_feature, population_enfants_situation):
    pop_totale = population_enfants_situation.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=admin_feature.geometry(),
        scale=population_enfants_situation.projection().nominalScale(),
        maxPixels=1e13
    ).get('b1')  # 'b1' est la bande par défaut pour les données de population

    # Créer un Feature avec l'ID de la zone et le nombre d'enfants atteints
    return ee.Feature(None, {
        'ID': admin_feature.get('shapeID'),  # 'shapeID' représente l'identifiant de la zone, à adapter selon le cas
        'Enfants_Atteints': pop_totale
    })

# Fonction pour convertir une FeatureCollection en dictionnaire pour extraire les ID et Enfants_Atteints
def features_to_dict(feature_collection):
    ids = feature_collection.aggregate_array('ID').getInfo()
    enfants_atteints = feature_collection.aggregate_array('Enfants_Atteints').getInfo()
    return [{'ID': id, 'Enfants_Atteints': enfants} for id, enfants in zip(ids, enfants_atteints)]

# Calculer la population d'enfants atteints pour chaque situation et chaque niveau administratif
# Aucune situation grave
population_enfants_aucune_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
population_enfants_aucune_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
#population_enfants_aucune_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
#population_enfants_aucune_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))

# Situation modérée
population_enfants_moderee_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
population_enfants_moderee_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
#population_enfants_moderee_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
#population_enfants_moderee_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))

# Crise
population_enfants_crise_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
population_enfants_crise_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
#population_enfants_crise_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
#population_enfants_crise_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))

# Extraire et exporter les données pour chaque niveau administratif et chaque situation

# Aucune situation grave
data_aucune_pays = features_to_dict(population_enfants_aucune_pays)
data_aucune_regions = features_to_dict(population_enfants_aucune_regions)
#data_aucune_departements = features_to_dict(population_enfants_aucune_departements)
#data_aucune_communes = features_to_dict(population_enfants_aucune_communes)

# Situation modérée
data_moderee_pays = features_to_dict(population_enfants_moderee_pays)
data_moderee_regions = features_to_dict(population_enfants_moderee_regions)
#data_moderee_departements = features_to_dict(population_enfants_moderee_departements)
#data_moderee_communes = features_to_dict(population_enfants_moderee_communes)

# Crise
data_crise_pays = features_to_dict(population_enfants_crise_pays)
data_crise_regions = features_to_dict(population_enfants_crise_regions)
#data_crise_departements = features_to_dict(population_enfants_crise_departements)
#data_crise_communes = features_to_dict(population_enfants_crise_communes)

# Convertir les données en DataFrames et exporter chaque DataFrame en CSV
# Aucune situation grave
pd.DataFrame(data_aucune_pays).to_csv('population_enfants_aucune_situation_pays.csv', index=False)
pd.DataFrame(data_aucune_regions).to_csv('population_enfants_aucune_situation_regions.csv', index=False)
#pd.DataFrame(data_aucune_departements).to_csv('population_enfants_aucune_situation_departements.csv', index=False)
#pd.DataFrame(data_aucune_communes).to_csv('population_enfants_aucune_situation_communes.csv', index=False)

# Situation modérée
pd.DataFrame(data_moderee_pays).to_csv('population_enfants_moderee_pays.csv', index=False)
pd.DataFrame(data_moderee_regions).to_csv('population_enfants_moderee_regions.csv', index=False)
#pd.DataFrame(data_moderee_departements).to_csv('population_enfants_moderee_departements.csv', index=False)
#pd.DataFrame(data_moderee_communes).to_csv('population_enfants_moderee_communes.csv', index=False)

# Crise
pd.DataFrame(data_crise_pays).to_csv('population_enfants_crise_pays.csv', index=False)
pd.DataFrame(data_crise_regions).to_csv('population_enfants_crise_regions.csv', index=False)
#pd.DataFrame(data_crise_departements).to_csv('population_enfants_crise_departements.csv', index=False)
#pd.DataFrame(data_crise_communes).to_csv('population_enfants_crise_communes.csv', index=False)

print("Les fichiers CSV pour chaque situation et niveau administratif ont été exportés avec succès.")

Les fichiers CSV pour chaque situation et niveau administratif ont été exportés avec succès.


In [60]:
#Affichons par exemple les valeurs obtenues pour aucune situation 
pd.DataFrame(data_aucune_regions)

,ID,Enfants_Atteints
0,87150794B58281898249288,661.991532
1,87150794B47488127174201,433.966333
2,87150794B29979307394387,119.439622
3,87150794B24708462658640,531.160082
4,87150794B5839560144768,766.066661
5,87150794B17606801029036,863.694865
